<h1>cnt_access, cnt_result, cnt_printを予測する</h1>

In [1]:
#必要なモジュールのimport
import pandas as pd
import numpy as np
import MeCab
from gensim.models import word2vec
import re
from io import StringIO
from joblib import Parallel, delayed
import pickle

from sklearn import preprocessing as sp
from sklearn.ensemble import RandomForestRegressor

In [2]:
#形態要素解析をし、learnW2Vmodelで学習したモデルを使って特徴量を計算するファンクション
def vectorize(docs, model):
    
    try:
        
        mecab = MeCab.Tagger("-Ochasen")
        data = mecab.parse(docs)
        data = StringIO(data.replace("\"", ""))
        data = pd.read_csv(data, sep='\t', header=None)
        data = data.loc[(data[3].str.find("名詞") >= 0) & (data[3].str.find("接頭詞") < 0) & (data[3].str.find("サ変接続") < 0)]
        
        #learnW2Vmodelで学習したボキャブラリーにない単語は無視する
        data = data[data[0].isin(pd.DataFrame.from_dict(model.wv.vocab, orient='index').index)]
        data = data[[0]]
        
        #１プレス発表の特徴量は、すべての単語の特徴量の平均と定義する
        vec = pd.DataFrame(data.applymap(model.wv.word_vec)[0].values.mean(axis=0))
        
        return vec
    
    except:
        
        return []

In [3]:
#vectorizeを使って特徴量の計算結果を返すファンクション
def getPressvec(df, model):

    #データフレームのテキストデータを読み込む
    df['text'] = df['title'] + "\n" + df['summary'] + "\n" + df['content']
    
    #vectorizeを使って特徴量を算出
    pressvec = pd.concat(Parallel(n_jobs=2, verbose=10)([delayed(vectorize)(docs, model) for docs in df['text'].str.lower().values]), axis=1).T
    pressvec.index = df.index

    return pressvec

In [4]:
#予測対象データとW2Vモデル、RFモデル（３種）を読み込み
df = pd.read_csv("target.csv")
model = word2vec.Word2Vec.load("word2vec.gensim.model")
RFmodels = [pickle.load(open("pred_cnt_access_model.pkl", "rb")),
           pickle.load(open("pred_cnt_result_model.pkl", "rb")),
           pickle.load(open("pred_cnt_print_model.pkl", "rb"))]

In [5]:
#データ確認
df.head()

,cnt_access,cnt_result,cnt_print,title,summary,content,send_date,genre_id,category_id
0,NaN,NaN,NaN,【真空管ドールズ】事前登録30 万人突破記念キャンペーン！告知公式Twitter フォロワー...,株式会社ソニー・ミュージックエンタテインメント(本社：東京都千代田区、代表取締役：水野 道訓...,株式会社ソニー・ミュージックエンタテインメント(本社：東京都千代田区、代表取締役：水野 道訓...,2016-06-13 10:00:00,23,8
1,NaN,NaN,NaN,コードを一切使わないプログラミング入門書『教えて♪プログラミング』の人気に応え無料フォローア...,コードを一切使わないプログラミング入門書 『教えて♪プログラミング――まずはなんとなく分かる...,●概要\n書名：『教えて♪プログラミング――まずはなんとなく分かることが大切よ！』\n高橋与...,2016-07-13 23:00:00,20,2
2,NaN,NaN,NaN,「温度」「湿度」「気圧」「照度」「騒音」「人の動き」施設内のデータ取得＆分析に適した月額課金...,インターネットプロダクト、ウェブサービス、人工知能などをIoT（Internet of Th...,インターネットプロダクト、ウェブサービス、人工知能などをIoT（Internet of Th...,2016-06-13 12:00:00,35,1
3,NaN,NaN,NaN,ビブリオバトルの映画版「Filmeet Award（フィルミートアワード）」、6月25日（土...,お勧め映画をプレゼンし合う「Filmeet Award(フィルミートアワード)」。6月のテー...,映画イベントを運営するFilmeet（フィルミート）は、6月25日（土）、湘南にある海の家「...,2016-06-13 16:00:00,24,6
4,NaN,NaN,NaN,9年連続日本一のデータ復旧業者デジタルデータリカバリーが、サーバ内に20台ほど立てていた仮想...,デジタルデータリカバリー（デジタルデータソリューション株式会社、本社：東京都中央区、代表取締...,【障害の状況】\n今回ご相談頂いた法人様はデータセンターのサービスを利用されていらっしゃり、...,2016-06-13 11:00:00,35,99


In [6]:
#特徴量計算(時間がかかります)
pressvec = getPressvec(df, model)  

[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Batch computation too fast (0.1789s.) Setting batch_size=2.
[Parallel(n_jobs=2)]: Done   3 out of   6 | elapsed:    0.4s remaining:    0.4s
[Parallel(n_jobs=2)]: Done   4 out of   6 | elapsed:    0.5s remaining:    0.3s
[Parallel(n_jobs=2)]: Done   6 out of   6 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   6 out of   6 | elapsed:    0.6s finished


In [7]:
#データ確認
pressvec.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.160970,-0.168390,0.051643,0.116799,-0.298749,-0.012004,0.356229,-0.007418,0.184713,-0.526594,...,0.395479,-0.571077,0.178304,-0.507863,-0.018256,-0.347540,0.027449,-0.163290,0.317629,-0.059529
1,0.124328,-0.189809,0.034846,0.111861,-0.100162,-0.011498,0.166440,-0.034903,0.105612,-0.184608,...,0.381010,-0.450194,0.214227,-0.461153,0.051047,-0.058889,0.074772,-0.078074,0.088992,-0.030309
2,0.113522,-0.332130,0.020866,-0.152669,-0.089544,0.184012,0.038888,-0.015820,-0.000373,-0.357240,...,0.447938,-0.514172,0.337736,-0.300922,0.183903,0.249023,0.050210,0.075683,-0.104121,-0.050429
3,0.258308,0.119922,0.419613,0.341128,-0.250887,-0.065163,0.214140,-0.088318,0.211288,-0.435874,...,0.005870,-0.555945,0.187030,-0.234942,0.011569,-0.584926,0.311704,0.114277,0.738257,-0.008528
4,0.023502,-0.276670,-0.084633,-0.104904,-0.339480,0.474368,0.260968,-0.160962,-0.057524,-0.645493,...,0.711851,-0.597852,0.263696,-0.653267,-0.163579,-0.150684,-0.117127,0.016193,-0.092830,0.152397


In [8]:
#カレンダ情報を説明変数に加える
df['send_date'] = pd.to_datetime(df.send_date)
df['year'] = df.send_date.dt.year
df['month'] = df.send_date.dt.month
df['day'] = df.send_date.dt.day
df['week'] = df.send_date.dt.week
df['weekday'] = df.send_date.dt.weekday

In [9]:
#特徴量のデータフレームに元データの説明変数を加える
pressvec = pd.concat([pressvec, df[['genre_id', 'category_id', 'year', 'month', 'day', 'week', 'weekday']]], axis=1)

In [10]:
#データ確認
pressvec.head()

,0,1,2,3,4,5,6,7,8,9,...,97,98,99,genre_id,category_id,year,month,day,week,weekday
0,0.160970,-0.168390,0.051643,0.116799,-0.298749,-0.012004,0.356229,-0.007418,0.184713,-0.526594,...,-0.163290,0.317629,-0.059529,23,8,2016,6,13,24,0
1,0.124328,-0.189809,0.034846,0.111861,-0.100162,-0.011498,0.166440,-0.034903,0.105612,-0.184608,...,-0.078074,0.088992,-0.030309,20,2,2016,7,13,28,2
2,0.113522,-0.332130,0.020866,-0.152669,-0.089544,0.184012,0.038888,-0.015820,-0.000373,-0.357240,...,0.075683,-0.104121,-0.050429,35,1,2016,6,13,24,0
3,0.258308,0.119922,0.419613,0.341128,-0.250887,-0.065163,0.214140,-0.088318,0.211288,-0.435874,...,0.114277,0.738257,-0.008528,24,6,2016,6,13,24,0
4,0.023502,-0.276670,-0.084633,-0.104904,-0.339480,0.474368,0.260968,-0.160962,-0.057524,-0.645493,...,0.016193,-0.092830,0.152397,35,99,2016,6,13,24,0


In [11]:
#3つの目的変数を予測する
for i in range(3):
    RFmodel = RFmodels[i]
    df[df.columns[i]] = RFmodel.predict(pressvec.values)

In [12]:
#データ確認
df.head()

,cnt_access,cnt_result,cnt_print,title,summary,content,send_date,genre_id,category_id,text,year,month,day,week,weekday
0,32.4,93.6,19.7,【真空管ドールズ】事前登録30 万人突破記念キャンペーン！告知公式Twitter フォロワー...,株式会社ソニー・ミュージックエンタテインメント(本社：東京都千代田区、代表取締役：水野 道訓...,株式会社ソニー・ミュージックエンタテインメント(本社：東京都千代田区、代表取締役：水野 道訓...,2016-06-13 10:00:00,23,8,【真空管ドールズ】事前登録30 万人突破記念キャンペーン！告知公式Twitter フォロワー...,2016,6,13,24,0
1,124.5,107.1,6.7,コードを一切使わないプログラミング入門書『教えて♪プログラミング』の人気に応え無料フォローア...,コードを一切使わないプログラミング入門書 『教えて♪プログラミング――まずはなんとなく分かる...,●概要\n書名：『教えて♪プログラミング――まずはなんとなく分かることが大切よ！』\n高橋与...,2016-07-13 23:00:00,20,2,コードを一切使わないプログラミング入門書『教えて♪プログラミング』の人気に応え無料フォローア...,2016,7,13,28,2
2,250.4,103.5,14.4,「温度」「湿度」「気圧」「照度」「騒音」「人の動き」施設内のデータ取得＆分析に適した月額課金...,インターネットプロダクト、ウェブサービス、人工知能などをIoT（Internet of Th...,インターネットプロダクト、ウェブサービス、人工知能などをIoT（Internet of Th...,2016-06-13 12:00:00,35,1,「温度」「湿度」「気圧」「照度」「騒音」「人の動き」施設内のデータ取得＆分析に適した月額課金...,2016,6,13,24,0
3,42.3,160.0,13.3,ビブリオバトルの映画版「Filmeet Award（フィルミートアワード）」、6月25日（土...,お勧め映画をプレゼンし合う「Filmeet Award(フィルミートアワード)」。6月のテー...,映画イベントを運営するFilmeet（フィルミート）は、6月25日（土）、湘南にある海の家「...,2016-06-13 16:00:00,24,6,ビブリオバトルの映画版「Filmeet Award（フィルミートアワード）」、6月25日（土...,2016,6,13,24,0
4,22.9,93.4,1.7,9年連続日本一のデータ復旧業者デジタルデータリカバリーが、サーバ内に20台ほど立てていた仮想...,デジタルデータリカバリー（デジタルデータソリューション株式会社、本社：東京都中央区、代表取締...,【障害の状況】\n今回ご相談頂いた法人様はデータセンターのサービスを利用されていらっしゃり、...,2016-06-13 11:00:00,35,99,9年連続日本一のデータ復旧業者デジタルデータリカバリーが、サーバ内に20台ほど立てていた仮想...,2016,6,13,24,0


In [13]:
#データ保存
df.to_csv("pred_result.csv", index=False)